In [1]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')

from sublimpy import turbulence
import matplotlib.pyplot as plt
from sublimpy import tidy
import metpy.constants

# Open data

## Open precipitation dataset

In [2]:
precip_file = "/storage/elilouis/sublimationofsnow/tilden_precip_data/kettle_ponds_precip.csv"

precip_df = pd.read_csv(precip_file)

precip_df['date'] = pd.to_datetime(precip_df['date'])

acc_precip_on_first_day = precip_df.set_index('date').loc['20221130'].acc_prec

precip_df = precip_df.set_index('date').loc['20221130': '20230510'].reset_index()
precip_df['acc_prec']  = precip_df['acc_prec'] - acc_precip_on_first_day

## Open SOS Measurement Dataset

In [5]:
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df_5Min = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet')
tidy_df_30Min = pd.read_parquet(f'tidy_df_30min_{start_date}_{end_date}_noplanar_fit_clean.parquet')
# convert time column to datetime
tidy_df_5Min['time'] = pd.to_datetime(tidy_df_5Min['time'])
tidy_df_30Min['time'] = pd.to_datetime(tidy_df_30Min['time'])
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df_5Min = tidy_df_5Min.set_index('time').sort_index().loc[start_date:end_date].reset_index()
tidy_df_30Min = tidy_df_30Min.set_index('time').sort_index().loc[start_date:end_date].reset_index()

In [6]:
# quick way to get variable info if we want it 
# import xarray as xr
# ds = xr.open_dataset("/storage/elilouis/sublimationofsnow/sosnoqc/isfs_20221228.nc")
# ds['SWE_p2_c']

# Calculate measured daily cumulative sublimation (for plotting)

In [8]:
tidy_df_daily = tidy_df_5Min[tidy_df_5Min.measurement == 'Cumulative sublimation measured'].set_index('time').groupby(
    [pd.Grouper(freq='1440Min'), 'tower', 'height', 'measurement', 'variable']
).max().reset_index()

# Plot

In [10]:
measurements_chart = alt.Chart(
    tidy_df_daily.query("measurement == 'Cumulative sublimation measured'").query("height > 1")
).mark_line(opacity=1, strokeWidth=1).encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Cumulative sublimation (mm)").scale(domain=[-5, 60]),
    alt.Color("height:O").scale(scheme='turbo'),
    detail = 'variable:N',
    # tooltip = 'variable'
).properties(width = 250, height = 250)
measurements_chart

alt.Chart(...)

In [13]:

snowpillow_chart = alt.Chart(
    tidy_df_30Min.query("variable == 'SWE_p2_c'").dropna()
).transform_window(
    frame = [-48, 48],
    rolling_median = 'median(value)'
).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("rolling_median:Q").title(["Snow water", "equivalent (mm)"])
).properties(width = 250, height = 83)

# snowpillow_and_precip_chart = snowpillow_chart +\
# alt.Chart(
#     precip_df
# ).mark_line(strokeDash=[2,4]).encode(
#     alt.X('date:T'),
#     alt.Y("acc_prec")
# )

snowpillow_chart


alt.Chart(...)

In [17]:
figure2 = (
    snowpillow_chart &
    (measurements_chart)
).resolve_scale(color='independent')

figure2.save("cum-sub-and-snowpack-swe.png", ppi=200)
figure2

alt.VConcatChart(...)

In [24]:
table_src = tidy_df_daily[
    tidy_df_daily.time == tidy_df_daily.time.max()
].query("measurement == 'Cumulative sublimation measured'").query("height > 1")[
    ['tower', 'height', 'value']
]
table_src.round(1).sort_values('height')

,tower,height,value
1760,c,3.0,34.3
1765,d,3.0,32.6
1767,ue,3.0,29.7
1769,uw,3.0,34.1
1761,c,5.0,36.9
1762,c,10.0,40.5
1766,d,10.0,54.9
1768,ue,10.0,35.0
1770,uw,10.0,41.0
1763,c,15.0,29.7
